<a href="https://colab.research.google.com/github/EmilyKan/pycs4001/blob/main/%E7%AC%AC%E4%BA%8C%E6%AC%A1%E5%B0%88%E6%A1%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

test_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/space/test.csv")
train_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/space/train.csv")
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/space/sample_submission.csv')
test_ids = test_data["PassengerId"]

In [ ]:

cat = train_data.columns[train_data.dtypes == 'object']
num = train_data.columns[train_data.dtypes != 'object']

train_data['Transported'] = train_data['Transported'].map({True:1, False:0})


In [ ]:
df = pd.concat([train_data, test_data], axis=0).copy()
df1 = df.copy()
df1 = df1.drop(['Cabin','Name'], axis=1)
df2 = df1.copy()

In [ ]:

#整理passenger id 資料
df2['group'] = df2.apply(lambda row: row['PassengerId'][0:4], axis=1)
df2['number'] = df2.apply(lambda row: int(row['PassengerId'][5:7]), axis=1)
df2 = df2.drop(['PassengerId'], axis=1)
#把Transported換到最後一行
features = df2.columns.tolist()
features.remove('Transported')
features.append('Transported')
df2 = df2[features].copy()
print(df2)

     HomePlanet CryoSleep    Destination   Age    VIP  RoomService  FoodCourt  \
0        Europa     False    TRAPPIST-1e  39.0  False          0.0        0.0   
1         Earth     False    TRAPPIST-1e  24.0  False        109.0        9.0   
2        Europa     False    TRAPPIST-1e  58.0   True         43.0     3576.0   
3        Europa     False    TRAPPIST-1e  33.0  False          0.0     1283.0   
4         Earth     False    TRAPPIST-1e  16.0  False        303.0       70.0   
...         ...       ...            ...   ...    ...          ...        ...   
4272      Earth      True    TRAPPIST-1e  34.0  False          0.0        0.0   
4273      Earth     False    TRAPPIST-1e  42.0  False          0.0      847.0   
4274       Mars      True    55 Cancri e   NaN  False          0.0        0.0   
4275     Europa     False            NaN   NaN  False          0.0     2680.0   
4276      Earth      True  PSO J318.5-22  43.0  False          0.0        0.0   

      ShoppingMall     Spa 

In [ ]:
#將資料分類
num_missing_features = df2.columns[5:-1].to_list()
num_missing_features.append('Age')

cat_missing_features = df2.columns[:5].to_list()
cat_missing_features.remove('Age')



In [ ]:
#標準化資料
from sklearn.preprocessing import StandardScaler

df3 = df2.copy()

scaler = StandardScaler()
df3[num_missing_features] = scaler.fit_transform(df3[num_missing_features])

#填充缺失值
from sklearn.impute import KNNImputer

df4 = df3.copy()

imputer = KNNImputer(n_neighbors=3)
df4[num_missing_features] = imputer.fit_transform(df4[num_missing_features])
df5 = df4.copy()

for feature in cat_missing_features:
    df5[feature] = df5[feature].fillna(value=df5[feature].mode()[0])


df5.isnull().sum()



df6 = df5.copy()

for feature in ['HomePlanet','Destination']:
    df6 = pd.get_dummies(df6, columns=[feature])
    df6 = df6.drop([df6.columns[-1]], axis=1)

for feature in ['CryoSleep','VIP']:
    df6[feature] = df6[feature].map({True:1, False:0})



In [ ]:
X_train = df6.iloc[:train_data.shape[0],:].copy()
X_train = X_train.drop(['Transported'], axis=1).copy()
y_train = train_data['Transported'].copy()
X_test = df6.iloc[train_data.shape[0]:,:]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
df.fillna(0) 
RF = RandomForestClassifier(n_estimators = 200,
                            max_depth = 30,
                            max_features = 4,
                            min_samples_split = 10,
                            min_samples_leaf = 10,
                            random_state = 0)
RF_cv_scores = cross_validate(RF, X_train, y_train, scoring='accuracy', cv=5)
round(RF_cv_scores['test_score'].mean(),5)


0.77557

In [ ]:
RF.fit(X_train, y_train)

RandomForestClassifier(max_depth=30, max_features=4, min_samples_leaf=10,
                       min_samples_split=10, n_estimators=200, random_state=0)

In [ ]:


y_pred = RF.predict(X_test.drop(['Transported'],axis=1))
submission['Transported'] = y_pred
submission['Transported'] = submission['Transported'].map({1:True, 0:False})
submission.head()



,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [ ]:


submission.to_csv('submission.csv', index=False)

